In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely.wkt
import pandas as pd
from sklearn import preprocessing
import time
from shapely.geometry import Point
from dateutil.relativedelta import relativedelta

In [2]:
crime_data = pd.read_csv('Crimes.csv')
weather_data = pd.read_csv('chicago_weather_data.csv')
weather_data = weather_data[weather_data['NAME'] == 'CHICAGO OHARE INTERNATIONAL AIRPORT, IL US']

In [3]:
weather_data = weather_data[['STATION', 'NAME', 'DATE', 'TAVG', 'TMAX', 'TMIN']]

In [4]:
crime_data['Primary Type'].value_counts()

THEFT                                191437
BATTERY                              148503
CRIMINAL DAMAGE                       85956
ASSAULT                               59239
DECEPTIVE PRACTICE                    56346
OTHER OFFENSE                         51581
BURGLARY                              37511
NARCOTICS                             37258
ROBBERY                               32114
MOTOR VEHICLE THEFT                   32077
CRIMINAL TRESPASS                     20288
WEAPONS VIOLATION                     14702
OFFENSE INVOLVING CHILDREN             6801
CRIM SEXUAL ASSAULT                    4852
PUBLIC PEACE VIOLATION                 4450
INTERFERENCE WITH PUBLIC OFFICER       3547
SEX OFFENSE                            3232
PROSTITUTION                           2134
HOMICIDE                               1988
ARSON                                  1253
LIQUOR LAW VIOLATION                    644
STALKING                                608
KIDNAPPING                      

Import block group & join with crimes

In [5]:
def process_blocks():
    df = pd.read_csv('blocks.csv')
    df['GEOID10'] = df['GEOID10'].astype(str)
    df["block_group"] = df["GEOID10"].apply(lambda block: block[:12])
    df["the_geom"] = df["the_geom"].apply(shapely.wkt.loads)
    gdf = gpd.GeoDataFrame(df).set_geometry("the_geom").drop(columns=df.columns.difference(["block_group", "the_geom"]))
    gdf = gpd.GeoDataFrame(gdf).set_geometry('the_geom')
    return gdf

In [6]:
blocks_df = process_blocks()

In [7]:
crime_data = crime_data[crime_data['Latitude'].notnull()]
crime_data['the_geom'] = crime_data.apply(lambda row: Point(float(row["Longitude"]), float(row["Latitude"])), axis=1)
crimes_gdf = gpd.GeoDataFrame(crime_data).set_geometry('the_geom')

In [8]:
joined = gpd.sjoin(crimes_gdf, blocks_df, how="left", op='intersects')
joined.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,the_geom,index_right,block_group
0,11706019,JC285982,05/31/2019 12:00:00 AM,028XX N TALMAN AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,1158221.0,1918668.0,2019,06/30/2019 03:56:27 PM,41.932573,-87.693969,"(41.932572626, -87.693969414)",POINT (-87.69396941399999 41.932572626),9664.0,170312109002
1,11709914,JC290748,05/31/2019 12:00:00 AM,014XX N WELLS ST,0890,THEFT,FROM BUILDING,BAR OR TAVERN,False,False,...,1174441.0,1909982.0,2019,06/30/2019 03:56:27 PM,41.908390,-87.634623,"(41.908390488, -87.634623176)",POINT (-87.63462317600001 41.908390488),11298.0,170310803004
2,11707466,JC287762,05/31/2019 12:00:00 AM,058XX W LAKE ST,1320,CRIMINAL DAMAGE,TO VEHICLE,ALLEY,False,False,...,1137312.0,1902246.0,2019,06/30/2019 03:56:27 PM,41.887911,-87.771204,"(41.88791087, -87.771204147)",POINT (-87.77120414700001 41.88791087),34793.0,170312514004
3,11730612,JC315728,05/31/2019 12:00:00 AM,012XX W 96TH ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,OTHER,False,False,...,1169672.0,1840995.0,2019,06/30/2019 03:56:27 PM,41.719188,-87.654145,"(41.71918826, -87.654144938)",POINT (-87.65414493799999 41.71918826),19851.0,170317304002
4,11706023,JC285978,05/31/2019 12:00:00 AM,020XX W 75TH PL,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,1164033.0,1854670.0,2019,06/30/2019 03:56:27 PM,41.756835,-87.674415,"(41.756834762, -87.674415394)",POINT (-87.67441539399999 41.756834762),44359.0,170317104006


In [9]:
joined['Date'] = joined['Date'].str.split(' ',expand=True)[0]

In [10]:
joined['Date'] = pd.to_datetime(joined['Date'])
weather_data['Date'] = pd.to_datetime(weather_data['DATE'])

In [11]:
combined = pd.merge(joined, weather_data, on='Date', how='outer')
combined

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Location,the_geom,index_right,block_group,STATION,NAME,DATE,TAVG,TMAX,TMIN
0,11706019.0,JC285982,2019-05-31,028XX N TALMAN AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,"(41.932572626, -87.693969414)",POINT (-87.69396941399999 41.932572626),9664.0,170312109002,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
1,11709914.0,JC290748,2019-05-31,014XX N WELLS ST,0890,THEFT,FROM BUILDING,BAR OR TAVERN,False,False,...,"(41.908390488, -87.634623176)",POINT (-87.63462317600001 41.908390488),11298.0,170310803004,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
2,11707466.0,JC287762,2019-05-31,058XX W LAKE ST,1320,CRIMINAL DAMAGE,TO VEHICLE,ALLEY,False,False,...,"(41.88791087, -87.771204147)",POINT (-87.77120414700001 41.88791087),34793.0,170312514004,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
3,11730612.0,JC315728,2019-05-31,012XX W 96TH ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,OTHER,False,False,...,"(41.71918826, -87.654144938)",POINT (-87.65414493799999 41.71918826),19851.0,170317304002,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
4,11706023.0,JC285978,2019-05-31,020XX W 75TH PL,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,"(41.756834762, -87.674415394)",POINT (-87.67441539399999 41.756834762),44359.0,170317104006,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
5,11706099.0,JC286075,2019-05-31,018XX S TROY ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,"(41.856347317, -87.704170333)",POINT (-87.70417033299999 41.856347317),35172.0,170318431002,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
6,11710038.0,JC291026,2019-05-31,050XX S DREXEL BLVD,1750,OFFENSE INVOLVING CHILDREN,CHILD ABUSE,APARTMENT,False,True,...,"(41.803653573, -87.603772447)",POINT (-87.60377244700001 41.80365357300001),5157.0,170313904002,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
7,11706995.0,JC286850,2019-05-31,028XX W FLOURNOY ST,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,"(41.872868511, -87.697336894)",POINT (-87.697336894 41.872868511),38069.0,170312718001,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
8,11706653.0,JC286400,2019-05-31,017XX W TAYLOR ST,1750,OFFENSE INVOLVING CHILDREN,CHILD ABUSE,HOSPITAL BUILDING/GROUNDS,False,True,...,"(41.869201889, -87.670589881)",POINT (-87.670589881 41.869201889),45876.0,170318382002,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-31,71.0,81.0,60.0
9,11705821.0,JC285616,2019-05-30,054XX S PAULINA ST,031A,ROBBERY,ARMED: HANDGUN,ALLEY,False,False,...,"(41.795311203, -87.66705)",POINT (-87.66705 41.795311203),16898.0,170316118001,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019-05-30,66.0,78.0,60.0


In [19]:
dfs_to_concat = [combined]
for col in ['Primary Type']:
    dummy_df = pd.get_dummies(combined[col])
    dfs_to_concat.append(dummy_df)
dataframe = pd.concat(dfs_to_concat, axis=1)

In [78]:
dataframe.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,OTHER NARCOTIC VIOLATION,OTHER OFFENSE,PROSTITUTION,PUBLIC INDECENCY,PUBLIC PEACE VIOLATION,ROBBERY,SEX OFFENSE,STALKING,THEFT,WEAPONS VIOLATION
0,11706019.0,JC285982,2019-05-31,028XX N TALMAN AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,0,0,0,0,0,0,0,0,0,0
1,11709914.0,JC290748,2019-05-31,014XX N WELLS ST,0890,THEFT,FROM BUILDING,BAR OR TAVERN,False,False,...,0,0,0,0,0,0,0,0,1,0
2,11707466.0,JC287762,2019-05-31,058XX W LAKE ST,1320,CRIMINAL DAMAGE,TO VEHICLE,ALLEY,False,False,...,0,0,0,0,0,0,0,0,0,0
3,11730612.0,JC315728,2019-05-31,012XX W 96TH ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,OTHER,False,False,...,0,0,0,0,0,0,0,0,0,0
4,11706023.0,JC285978,2019-05-31,020XX W 75TH PL,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,0,0,0,0,0,0,0,0,0,0


In [169]:
def get_crime_dict(crime_list):
    rt = {}
    for idx, crime in enumerate(crime_list):
        rt[crime] = idx
    return rt

def limit_by_crime_type(crime_list, df):
    return df[ df["Primary Type"].isin(crime_list)]

def replace_crime_type_with_id(df, crime_dict):
    df["Target"] = [0 for i in range(df.shape[0])]
    for idx, row in df.iterrows():
        c = row["Primary Type"]
        c_id = crime_dict[c]
        df.at[idx, "Target"] = c_id        
    return df

def main(data):
    types_of_crime = ["THEFT", "HOMICIDE", "BATTERY"]
    crime_dict = get_crime_dict(types_of_crime)
    df = limit_by_crime_type(types_of_crime, data)
    replaced_df = replace_crime_type_with_id(df, crime_dict)
    xs_list = ["Target", "TAVG", "Date", "Primary Type", "block_group"]
    model_df = replaced_df[xs_list]
    return model_df

In [170]:
df = main(dataframe)

/Users/katykoeing/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [173]:
def limit_blocks_by_count(df):

    blocks = {}
    for idx, row in df.iterrows():
        b = row["block_group"]
        if b not in blocks:
            blocks[b] = 1
        else:
            blocks[b] += 1
            
    keepers = {}
    for k, v in blocks.items():
        if v > 10:
            keepers[k] = v
    
    return keepers

b = limit_blocks_by_count(df)

In [174]:
blocks = list(b.keys())

limited_by_blocks = df[ df["block_group"].isin(blocks)]

In [177]:
binary = pd.get_dummies(limited_by_blocks, columns=["block_group"])

In [178]:
limited_by_blocks.head()

,Target,TAVG,Date,Primary Type,block_group
1,0,71.0,2019-05-31,THEFT,170310803004
12,2,66.0,2019-05-30,BATTERY,170314907003
18,2,66.0,2019-05-30,BATTERY,170318412001
19,2,66.0,2019-05-30,BATTERY,170314903002
20,2,66.0,2019-05-30,BATTERY,170311102001


In [180]:
binary.head()

,Target,TAVG,Date,Primary Type,block_group_170310101001,block_group_170310101002,block_group_170310101003,block_group_170310102011,block_group_170310102012,block_group_170310102013,...,block_group_170318437002,block_group_170318438001,block_group_170318438002,block_group_170318439001,block_group_170318439002,block_group_170318439003,block_group_170318439004,block_group_170318439005,block_group_170319800001,block_group_170319801001
1,0,71.0,2019-05-31,THEFT,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,2,66.0,2019-05-30,BATTERY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,2,66.0,2019-05-30,BATTERY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,2,66.0,2019-05-30,BATTERY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,2,66.0,2019-05-30,BATTERY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
binary.to_csv("sagemaker_data.csv", index=False)

Data Cleaning & Feature Generation

In [ ]:
dataframe.columns

grouped = dataframe.groupby(['Date', 'block_group'])[['Primary Type_HOMICIDE', 'Arrest_False',
       'Arrest_True', 'Domestic_False', 'Domestic_True', 'TAVG']].sum().reset_index()

scaler = preprocessing.MinMaxScaler()
scaler.fit(grouped[['TAVG']])
preprocessing.MinMaxScaler(copy=True, feature_range=(0, 1))
grouped['TAVG'] = scaler.transform(grouped[['TAVG']])
grouped.head()

grouped.to_csv('data_for_sagemaker.csv')